In [35]:
%matplotlib inline

In [90]:
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import ndimage

In [ ]:
# Making nice smooth cortex mask from the one of the previous notebook
!fslmaths projects/gradients/atlas/cortex_mask_25um.nii.gz -s 0.2 -thr 0.4 -bin cortex_mask_25um_smooth.nii.gz

In [ ]:
# Dilating this mask to create a fake white matter mask
!fslmaths cortex_mask_25um_smooth.nii.gz -dilM -dilM cortex_mask_25um_smooth_dilM2.nii.gz

In [121]:
# Improve the "allmask" from last notebook
allmask = nb.load("/home/julia/projects/gradients/atlas/make_cortex/all_mask_25um.nii.gz")
atlas = nb.load("/home/julia/projects/gradients/atlas/ABI_atlas_25um.nii.gz")

In [122]:
allmask_new = allmask.get_data()
new_labels = [61, 72, 73, 290, 301, 302]
for l in new_labels: 
    allmask_new[np.where(atlas.get_data()==l)]=1

In [123]:
nb.save(nb.Nifti1Image(allmask_new, allmask.affine, allmask.header), 
        "/home/julia/projects/gradients/atlas/make_cortex/all_mask_25um_v2.nii.gz")

In [124]:
cortex = nb.load("/home/julia/projects/gradients/atlas/cortex_mask_25um_smooth.nii.gz")
#cortex_dil = nb.load("/home/julia/projects/gradients/atlas/cortex_mask_25um_smooth_dilM2.nii.gz")

In [125]:
inside = allmask_new - cortex.get_data()
inside[np.where(inside==-1)]=0

In [126]:
nb.save(nb.Nifti1Image(inside, cortex.affine, cortex.header), 
        "/home/julia/projects/gradients/atlas/make_cortex/inside_mask_25um_v2.nii.gz")

In [127]:
wm_fake = np.zeros(cortex.get_data().shape)
wm_fake[np.where(inside>0)] = 1
wm_fake[np.where(cortex.get_data()>0)] = 0
wm_fake *= cortex_dil.get_data()

In [128]:
nb.save(nb.Nifti1Image(wm_fake, cortex.affine, cortex.header), 
        "/home/julia/projects/gradients/atlas/make_cortex/wm_fake_25um_v2.nii.gz")